# T1040 - Network Sniffing
Adversaries may sniff network traffic to capture information about an environment, including authentication material passed over the network. Network sniffing refers to using the network interface on a system to monitor or capture information sent over a wired or wireless connection. An adversary may place a network interface into promiscuous mode to passively access data in transit over the network, or use span ports to capture a larger amount of data.

Data captured via this technique may include user credentials, especially those sent over an insecure, unencrypted protocol. Techniques for name service resolution poisoning, such as [LLMNR/NBT-NS Poisoning and SMB Relay](https://attack.mitre.org/techniques/T1557/001), can also be used to capture credentials to websites, proxies, and internal systems by redirecting traffic to an adversary.

Network sniffing may also reveal configuration details, such as running services, version numbers, and other network characteristics (e.g. IP addresses, hostnames, VLAN IDs) necessary for subsequent Lateral Movement and/or Defense Evasion activities.

In cloud-based environments, adversaries may still be able to use traffic mirroring services to sniff network traffic from virtual machines. For example, AWS Traffic Mirroring, GCP Packet Mirroring, and Azure vTap allow users to define specified instances to collect traffic from and specified targets to send collected traffic to.(Citation: AWS Traffic Mirroring) (Citation: GCP Packet Mirroring) (Citation: Azure Virtual Network TAP) Often, much of this traffic will be in cleartext due to the use of TLS termination at the load balancer level to reduce the strain of encrypting and decrypting traffic.(Citation: Rhino Security Labs AWS VPC Traffic Mirroring) (Citation: SpecterOps AWS Traffic Mirroring) The adversary can then use exfiltration techniques such as Transfer Data to Cloud Account in order to access the sniffed traffic. (Citation: Rhino Security Labs AWS VPC Traffic Mirroring)

## Atomic Tests

### Atomic Test #1 - Packet Capture Linux using tshark or tcpdump
Perform a PCAP. Wireshark will be required for tshark. TCPdump may already be installed.

Upon successful execution, tshark or tcpdump will execute and capture 5 packets on interface ens33.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Check if at least one of tcpdump or tshark is installed.

##### Check Prereq Commands:
```bash
if [ ! -x "$(command -v tcpdump)" ] && [ ! -x "$(command -v tshark)" ]; then exit 1; else exit 0; fi;

```
##### Get Prereq Commands:
```bash
(which yum && yum -y install epel-release tcpdump tshark)||(which apt-get && DEBIAN_FRONTEND=noninteractive apt-get install -y tcpdump tshark)

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
tcpdump -c 5 -nnni ens33
tshark -c 5 -i ens33
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 1

### Atomic Test #2 - Packet Capture macOS using tcpdump or tshark
Perform a PCAP on macOS. This will require Wireshark/tshark to be installed. TCPdump may already be installed.

Upon successful execution, tshark or tcpdump will execute and capture 5 packets on interface en0A.

**Supported Platforms:** macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Check if at least one of tcpdump or tshark is installed.

##### Check Prereq Commands:
```bash
if [ ! -x "$(command -v tcpdump)" ] && [ ! -x "$(command -v tshark)" ]; then exit 1; else exit 0; fi;

```
##### Get Prereq Commands:
```bash
(which yum && yum -y install epel-release tcpdump tshark)||(which apt-get && DEBIAN_FRONTEND=noninteractive apt-get install -y tcpdump tshark)

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo tcpdump -c 5 -nnni en0A    
if [ -x "$(command -v tshark)" ]; then sudo tshark -c 5 -i en0A; fi;
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 2

### Atomic Test #3 - Packet Capture Windows Command Prompt
Perform a packet capture using the windows command prompt. This will require a host that has Wireshark/Tshark
installed.

Upon successful execution, tshark will execute and capture 5 packets on interface "Ethernet".

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: tshark must be installed and in the default path of "c:\Program Files\Wireshark\Tshark.exe".

##### Check Prereq Commands:
```cmd
if (test-path "c:\program files\wireshark\tshark.exe") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest -OutFile $env:temp\wireshark_installer.exe https://1.eu.dl.wireshark.org/win64/Wireshark-win64-latest.exe
Start-Process $env:temp\wireshark_installer.exe /S

```
##### Description: npcap must be installed.

##### Check Prereq Commands:
```cmd
if (test-path "C:\Program Files\Npcap\npcap.sys") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest -OutFile $env:temp\npcap_installer.exe https://nmap.org/npcap/dist/npcap-1.31.exe
Start-Process $env:temp\npcap_installer.exe

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"c:\Program Files\Wireshark\tshark.exe" -i Ethernet -c 5
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 3

### Atomic Test #4 - Windows Internal Packet CaptureUses the built-in Windows packet capture
After execution you should find a file named trace.etl and trace.cab in the temp directory**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
netsh trace start capture=yes tracefile=%temp%\trace.etl maxsize=10```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 4

#### Cleanup: 
```cmd
netsh trace stop >nul 2>&1
TIMEOUT /T 5 >nul 2>&1
del %temp%\trace.etl >nul 2>&1
del %temp%\trace.cab >nul 2>&1```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Windows Internal pktmon captureWill start a packet capture and store log file as t1040.etl.
https://lolbas-project.github.io/lolbas/Binaries/Pktmon/**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
pktmon.exe start --etw  -f %TEMP%\t1040.etl
TIMEOUT /T 5 >nul 2>&1
pktmon.exe stop
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 5

#### Cleanup: 
```cmd
del %TEMP%\t1040.etl```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Windows Internal pktmon set filterSelect Desired ports for packet capture 
https://lolbas-project.github.io/lolbas/Binaries/Pktmon/**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
pktmon.exe filter add -p 445
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 6

#### Cleanup: 
```cmd
pktmon filter remove```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Packet Capture macOS using /dev/bpfN with sudo
Opens a /dev/bpf file (O_RDONLY) and captures packets for a few seconds.

**Supported Platforms:** macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: compile C program

##### Check Prereq Commands:
```bash
exit 1

```
##### Get Prereq Commands:
```bash
cc PathToAtomicsFolder/T1040/src/macos_pcapdemo.c -o /tmp/t1040_macos_pcapdemo

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo /tmp/t1040_macos_pcapdemo -i en0 -t 3
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 7

#### Cleanup: 
```bash
rm -f /tmp/t1040_macos_pcapdemo
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 7 -Cleanup

### Atomic Test #8 - Filtered Packet Capture macOS using /dev/bpfN with sudo
Opens a /dev/bpf file (O_RDONLY), sets BPF filter for 'udp' and captures packets for a few seconds.

**Supported Platforms:** macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: compile C program

##### Check Prereq Commands:
```bash
exit 1

```
##### Get Prereq Commands:
```bash
cc PathToAtomicsFolder/T1040/src/macos_pcapdemo.c -o /tmp/t1040_macos_pcapdemo

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo /tmp/t1040_macos_pcapdemo -f -i en0 -t 3
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 8

#### Cleanup: 
```bash
rm -f /tmp/t1040_macos_pcapdemo
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Packet Capture Linux socket AF_PACKET,SOCK_RAW with sudo
Captures packets with domain=AF_PACKET, type=SOCK_RAW for a few seconds.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: compile C program

##### Check Prereq Commands:
```bash
if [ -f "/tmp/t1040_linux_pcapdemo" ]; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```bash
cc PathToAtomicsFolder/T1040/src/linux_pcapdemo.c -o /tmp/t1040_linux_pcapdemo

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo /tmp/t1040_linux_pcapdemo -a -t 3
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 9

#### Cleanup: 
```bash
rm -f /tmp/t1040_linux_pcapdemo
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Packet Capture Linux socket AF_INET,SOCK_RAW,TCP with sudo
Captures packets with domain=AF_INET,type=SOCK_RAW,protocol=TCP for a few seconds.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: compile C program

##### Check Prereq Commands:
```bash
if [ -f "/tmp/t1040_linux_pcapdemo" ]; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```bash
cc PathToAtomicsFolder/T1040/src/linux_pcapdemo.c -o /tmp/t1040_linux_pcapdemo

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo /tmp/t1040_linux_pcapdemo -4 -p 6 -t 3
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 10

#### Cleanup: 
```bash
rm -f /tmp/t1040_linux_pcapdemo
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 10 -Cleanup

### Atomic Test #11 - Packet Capture Linux socket AF_INET,SOCK_PACKET,UDP with sudo
Captures packets with domain=AF_INET,type=SOCK_PACKET,protocol=UDP for a few seconds.
SOCK_PACKET is "obsolete" according to the man page, but still works on Ubuntu 20.04

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: compile C program

##### Check Prereq Commands:
```bash
if [ -f "/tmp/t1040_linux_pcapdemo" ]; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```bash
cc PathToAtomicsFolder/T1040/src/linux_pcapdemo.c -o /tmp/t1040_linux_pcapdemo

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 11 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo /tmp/t1040_linux_pcapdemo -4 -P -p 17 -t 3
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 11

#### Cleanup: 
```bash
rm -f /tmp/t1040_linux_pcapdemo
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 11 -Cleanup

### Atomic Test #12 - Packet Capture Linux socket AF_PACKET,SOCK_RAW with BPF filter for UDP with sudo
Captures packets with domain=AF_PACKET,type=SOCK_RAW for a few seconds.
Sets a BPF filter on the socket to filter for UDP traffic.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: compile C program

##### Check Prereq Commands:
```bash
if [ -f "/tmp/t1040_linux_pcapdemo" ]; then exit 0; else exit 1; fi

```
##### Get Prereq Commands:
```bash
cc PathToAtomicsFolder/T1040/src/linux_pcapdemo.c -o /tmp/t1040_linux_pcapdemo

```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 12 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
sudo /tmp/t1040_linux_pcapdemo -a -f -t 3
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 12

#### Cleanup: 
```bash
rm -f /tmp/t1040_linux_pcapdemo
```

In [ ]:
Invoke-AtomicTest T1040 -TestNumbers 12 -Cleanup

## Detection
Detecting the events leading up to sniffing network traffic may be the best method of detection. From the host level, an adversary would likely need to perform a [Adversary-in-the-Middle](https://attack.mitre.org/techniques/T1557) attack against other devices on a wired network in order to capture traffic that was not to or from the current compromised system. This change in the flow of information is detectable at the enclave network level. Monitor for ARP spoofing and gratuitous ARP broadcasts. Detecting compromised network devices is a bit more challenging. Auditing administrator logins, configuration changes, and device images is required to detect malicious changes.

In cloud-based environments, monitor for the creation of new traffic mirrors or modification of existing traffic mirrors.


## Shield Active Defense
### Software Manipulation 
 Make changes to a system's software properties and functions to achieve a desired effect. 

 Software Manipulation allows a defender to alter or replace elements of the operating system, file system, or any other software installed and executed on a system.
#### Opportunity
There is an opportunity for the defender to observe the adversary and control what they can see, what effects they can have, and/or what data they can access.
#### Use Case
By changing the output of network sniffing utilities normally found on a system, you can prevent adversaries from seeing particular content or making use of the results at all.
#### Procedures
Hook the Win32 Sleep() function so that it always performs a Sleep(1) instead of the intended duration. This can increase the speed at which dynamic analysis can be performed when a normal malicious file sleeps for long periods before attempting additional capabilities.
Hook the Win32 NetUserChangePassword() and modify it such that the new password is different from the one provided. The data passed into the function is encrypted along with the modified new password, then logged so a defender can get alerted about the change as well as decrypt the new password for use.
Alter the output of an adversary's profiling commands to make newly-built systems look like the operating system was installed months earlier.
Alter the output of adversary recon commands to not show important assets, such as a file server containing sensitive data.
